# RAG system evaluation

In [17]:
import os
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

True

# Generate questions based in the documents

This questions will be used to evaluate the system later.

## Opening files

Unstructured is the best to open and clean pdf files. It also partition the file to better undestand its structure, but we dont use the partitions in this step.

In [3]:
from unstructured.partition.auto import partition

folder_path = "../docs"

path, dirs, files = next(os.walk(folder_path))
        
documents_all = []
for file in files:
    with open( os.path.join(path, file), 'rb') as f:
        print(f'Opening file: {file}')
        
        # open file, clean, segment
        elements = partition(file=f)
        
        doc = "\n".join( [element.text for element in elements] )
            
        documents_all.append(doc)

len( documents_all )

Opening file: Manual_de_utilização_do_PJe_sem_certificado_digital.pdf
Opening file: processo_judicial_eletronico_grafica2.pdf
Opening file: TRT04 - Guia para participação em audiências_sessões com a ferramenta Zoom.pdf
Opening file: TRT04 - PJE 2.10.1 - Manual de autuação no  painel do procurador.pdf
Opening file: guiapje-tribunal.pdf
Opening file: guiapje-usuarios.pdf
Opening file: Instalação do PJePortable v. 1.13.10 - para uso do PJe.pdf
Opening file: PJe Mídias Desktop - Envio de Perícias em MP4.pdf
Opening file: TRT04 - MANUAL PJE 2.10.1 - PAINEL DO ADVOGADO.pdf
Opening file: Zoom - CDTI OAB.pdf
Opening file: PJeMídias - Acesso à gravação de audiências ou perícias.pdf
Opening file: TRT04 - Manual para advogados e peritos (cadastramento de dados bancários para créditos em processos do TRT04).pdf
Opening file: PJeMídiasDesktop - Envio de Provas em MP4.pdf
Opening file: PROCEDIMENTOS PARA ENVIO DE CÁLCULOS DO PJECALC NO PJE PARA ADVOGADOS.pdf
Opening file: guiapje-advogados.pdf
Openi

19

In [4]:
from pydantic import BaseModel, Field

class QuestionSchema(BaseModel):
    question: str = Field(..., description="The actual question text.")

class QuestionListSchema(BaseModel):
    questionList: list[QuestionSchema] = Field(..., description="A list of Question")

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_generate_questions = (
    "I'm creating a RAG system that works with knowledge base from a company.\n"
    "You are expert in RAG systems.\n"
    "I need you to create a set of questions based in the document provided.\n"
    "This questions will be used to evaluate the system later.\n"
    "The questions should be similar to what a employee would ask in the system.\n"
    "This is very important for the future of the company and my carreer!\n"
    "Output format: You should output in JSON format considering the provided schema.\n"
    "QUANTITY OF QUESTIONS: {QUANTITY}\n"
    "DOCUMENT:\n\n {DOCUMENT}\n\n"
)

prompt_generate_questions_lc = PromptTemplate.from_template(prompt_generate_questions)

In [11]:
from langchain_openai import ChatOpenAI

llm_openai = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=500,
    api_key=os.getenv("OPENAI_API_KEY")
).with_structured_output(QuestionListSchema)

In [12]:
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain_generate_questions = (
    prompt_generate_questions_lc
    | llm_openai
)

In [13]:
responses = []
for doc in documents_all:
    response = chain_generate_questions.invoke({
        "DOCUMENT": doc,
        "QUANTITY": 5
    })
    responses.append(response)

In [14]:
questions = []
for response in responses:
    for question in response.questionList:
        q = {"question": question.question}
        questions.append(q)
len(questions)

95

In [16]:
questions

[{'question': 'Como posso acessar o PJe sem um certificado digital?'},
 {'question': 'Quais são as restrições para usuários que acessam o PJe sem certificado digital?'},
 {'question': 'Quais documentos são necessários para cadastrar um novo usuário sem certificado digital?'},
 {'question': 'Como um usuário existente com certificado digital pode criar uma senha de acesso ao PJe?'},
 {'question': 'Quais procedimentos devem ser seguidos pela secretaria do tribunal para cadastrar um usuário no PJe?'},
 {'question': 'Qual é o principal objetivo do sistema Processo Judicial Eletrônico (PJe)?'},
 {'question': 'Como o PJe pode reduzir o tempo necessário para chegar a uma decisão judicial?'},
 {'question': 'Quais são as principais mudanças na gestão dos tribunais com a implementação do PJe?'},
 {'question': 'De que maneira o PJe melhora a segurança e a liberdade dos usuários no sistema?'},
 {'question': 'Quais são os requisitos de infraestrutura tecnológica necessários para a instalação do PJe?

In [18]:
df_questions = pd.DataFrame(questions)
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  95 non-null     object
dtypes: object(1)
memory usage: 892.0+ bytes


In [19]:
df_questions.head()

,question
0,Como posso acessar o PJe sem um certificado di...
1,Quais são as restrições para usuários que aces...
2,Quais documentos são necessários para cadastra...
3,Como um usuário existente com certificado digi...
4,Quais procedimentos devem ser seguidos pela se...


In [21]:
df_questions.to_csv("df_questions.csv", index=False)

# System configuration

## Vector store provider

In [49]:
import weaviate
from weaviate.embedded import EmbeddedOptions

# client = weaviate.connect_to_embedded()

weaviate_client = weaviate.WeaviateClient(
    embedded_options=EmbeddedOptions(
        persistence_data_path="../weaviate_persistence3"
    )
)

weaviate_client

{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-10-15T19:19:11Z"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-10-15T19:19:11Z"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-10-15T19:19:11Z"}
{"level":"info","msg":"module offload-s3 is enabled","time":"2024-10-15T19:19:11Z"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-10-15T19:19:11Z"}
{"level":"info","msg":"open cluster service","servers":{"Embedded_at_8079":38347},"time":"2024-10-15T19:19:11Z"}
{"address":"172.17.0.2:38

{"action":"telemetry_push","level":"info","msg":"telemetry started","payload":"\u0026{MachineID:6e3a5ece-37ae-4f35-a9ee-46db3c96a43b Type:INIT Version:1.26.1 NumObjects:0 OS:linux Arch:amd64 UsedModules:[]}","time":"2024-10-15T19:19:14Z"}
{"action":"bootstrap","level":"info","msg":"node reporting ready, node has probably recovered cluster from raft config. Exiting bootstrap process","time":"2024-10-15T19:19:14Z"}


## Embedding provider

In [50]:
from langchain_community.embeddings import GPT4AllEmbeddings
        
model_name = "nomic-embed-text-v1.f16.gguf"

gpt4all_kwargs = {'allow_download': 'True'}

embeddings_provider = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
)
embeddings_provider

/workspaces/llm-rag-evaluation/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in GPT4AllEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


GPT4AllEmbeddings(model_name='nomic-embed-text-v1.f16.gguf', n_threads=None, device='cpu', gpt4all_kwargs={'allow_download': 'True'}, client=<gpt4all.gpt4all.Embed4All object at 0x7f49ed35e950>)

# File ingestion and chunking


- load pdf with pymuPDFLoader
    - partition 
- load with unstructured
    - use partion from unstrutured
- embbed
    - gemini
    - gpt4all
    - allmini (multilingual)
- create a idnex for each one

## Document loading

### Simple loading

In [31]:
from langchain.document_loaders import PyMuPDFLoader

folder_path = "../docs"

path, dirs, files = next(os.walk(folder_path))
        
documents_all = []
for file in files:
    file_path = os.path.join(path, file)
    print(f'Opening file: {file_path}')
    
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()

    document = "\n".join( [doc.page_content for doc in documents] )
    
    documents_all.append( (file_path, document) )

len(documents_all)

Opening file: ../docs/Manual_de_utilização_do_PJe_sem_certificado_digital.pdf
Opening file: ../docs/processo_judicial_eletronico_grafica2.pdf
Opening file: ../docs/TRT04 - Guia para participação em audiências_sessões com a ferramenta Zoom.pdf
Opening file: ../docs/TRT04 - PJE 2.10.1 - Manual de autuação no  painel do procurador.pdf
Opening file: ../docs/guiapje-tribunal.pdf
Opening file: ../docs/guiapje-usuarios.pdf
Opening file: ../docs/Instalação do PJePortable v. 1.13.10 - para uso do PJe.pdf


/workspaces/llm-rag-evaluation/.venv/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 22 of document ../docs/processo_judicial_eletronico_grafica2.pdf
  warnings.warn(
/workspaces/llm-rag-evaluation/.venv/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 3 of document ../docs/Zoom - CDTI OAB.pdf
  warnings.warn(
/workspaces/llm-rag-evaluation/.venv/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 4 of document ../docs/Zoom - CDTI OAB.pdf
  warnings.warn(
/workspaces/llm-rag-evaluation/.venv/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 5 of document ../docs/Zoom - CDTI OAB.pdf
  warnings.warn(
/workspaces/llm-rag-evaluation/.venv/lib/python3.11/site-packages/langcha

Opening file: ../docs/PJe Mídias Desktop - Envio de Perícias em MP4.pdf
Opening file: ../docs/TRT04 - MANUAL PJE 2.10.1 - PAINEL DO ADVOGADO.pdf
Opening file: ../docs/Zoom - CDTI OAB.pdf
Opening file: ../docs/PJeMídias - Acesso à gravação de audiências ou perícias.pdf
Opening file: ../docs/TRT04 - Manual para advogados e peritos (cadastramento de dados bancários para créditos em processos do TRT04).pdf
Opening file: ../docs/PJeMídiasDesktop - Envio de Provas em MP4.pdf
Opening file: ../docs/PROCEDIMENTOS PARA ENVIO DE CÁLCULOS DO PJECALC NO PJE PARA ADVOGADOS.pdf
Opening file: ../docs/guiapje-advogados.pdf
Opening file: ../docs/Cadastramento inicial no PJe.pdf
Opening file: ../docs/MANUAL FUNCIONAMENTO DAS PROCURADORIAS NO PJE.pdf
Opening file: ../docs/Manual.pdf
Opening file: ../docs/PJE - Processo Judicial Eletrônico.pdf


19

In [33]:
documents_all[:3]

[('../docs/Manual_de_utilização_do_PJe_sem_certificado_digital.pdf',
  '\xa0\n \n\xa0\n\nManual de utilização do PJe sem certificado \ndigital \n\xa0\nÍndice \n\xa0\nIntrodução\xa0\nRestrições\xa0de\xa0acesso\xa0ao\xa0usuário\xa0que\xa0acessar\xa0o\xa0PJe\xa0sem\xa0certificado\xa0digital\xa0\nProcedimento\xa0para\xa0criar\xa0uma\xa0senha\xa0de\xa0acesso\xa0ao\xa0sistema\xa0sem\xa0certificado\xa0digital\xa0\nNovos\xa0usuários\xa0do\xa0sistema\xa0\nUsuário\xa0que\xa0já\xa0possuam\xa0um\xa0certificado\xa0digital\xa0\nUsuário\xa0sem\xa0certificado\xa0digital\xa0\nUsuário\xa0já\xa0existente\xa0com\xa0certificado\xa0digital\xa0\nUsuário\xa0cadastrado\xa0como\xa0Push,\xa0sem\xa0certificado\xa0digital\xa0\nProcedimento\xa0de\xa0secretaria\xa0nos\xa0tribunais\xa0\nPossíveis\xa0problemas\xa0e\xa0respostas\xa0prováveis\xa0\nProcedimentos\xa0para\xa0o\xa0administrador\xa0do\xa0sistema\xa0\nHabilitação\xa0ou\xa0restrição\xa0da\xa0funcionalidade\xa0de\xa0acesso\xa0sem\xa0o\xa0certificado\xa0digital\

### Simple chunking

Just for clarification, below is given examples of chunk_size.

In [28]:
text = """
Guia
do
Rápido
  Pendentes de ciência ou de seu registro: conjunto de atos de comunicação em relação ao qual ainda 
não há registro de ciência pelo destinatário, independentemente do meio de intimação utilizado. Quando 
se tratar de citação ou notificação eletrônica na forma da Lei n.º 11.419/2006, é incluída a informação do 
prazo máximo para ciência pelo destinatário, após o que ocorrerá a ciência ficta. O ícone “lupa vermelha” 
permite que o destinatário tome ciência do ato. O ícone de resposta levará à ciência e a uma página em que 
a resposta pode ser elaborada.
  Ciência dada pelo destinatário direto ou indireto e dentro do prazo: conjunto de atos de comu­
nicação em relação ao qual houve ciência pelo destinatário ou por quem o represente, independentemente 
do meio de intimação utilizado, e cujo prazo ainda está em curso. Caso o ato não tenha prazo para resposta, 
será exibido no agrupador “Sem prazo”. O prazo limite provável é exibido no canto inferior direito, assim como 
"""

print(f"Characters count: {len(text)}")

Characters count: 997


In [37]:
metadata_ = [ {"filename": doc[0] } for doc in documents_all ]
metadata_[:5]

[{'filename': '../docs/Manual_de_utilização_do_PJe_sem_certificado_digital.pdf'},
 {'filename': '../docs/processo_judicial_eletronico_grafica2.pdf'},
 {'filename': '../docs/TRT04 - Guia para participação em audiências_sessões com a ferramenta Zoom.pdf'},
 {'filename': '../docs/TRT04 - PJE 2.10.1 - Manual de autuação no  painel do procurador.pdf'},
 {'filename': '../docs/guiapje-tribunal.pdf'}]

In [38]:
from langchain_text_splitters import CharacterTextSplitter

characterTextSplitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [47]:
docs = [doc[1] for doc in documents_all] # take the doc out form the tuple
documents_lc = characterTextSplitter.create_documents( docs, metadatas=metadata_ )

Created a chunk of size 1042, which is longer than the specified 1000
Created a chunk of size 2791, which is longer than the specified 1000
Created a chunk of size 1673, which is longer than the specified 1000
Created a chunk of size 1306, which is longer than the specified 1000
Created a chunk of size 1880, which is longer than the specified 1000
Created a chunk of size 1054, which is longer than the specified 1000
Created a chunk of size 2112, which is longer than the specified 1000
Created a chunk of size 1228, which is longer than the specified 1000
Created a chunk of size 1423, which is longer than the specified 1000
Created a chunk of size 2737, which is longer than the specified 1000
Created a chunk of size 2492, which is longer than the specified 1000
Created a chunk of size 1688, which is longer than the specified 1000
Created a chunk of size 1702, which is longer than the specified 1000
Created a chunk of size 1028, which is longer than the specified 1000
Created a chunk of s

In [48]:
print(documents_lc[0])

page_content='Manual de utilização do PJe sem certificado 
digital 
 
Índice 
 
Introdução 
Restrições de acesso ao usuário que acessar o PJe sem certificado digital 
Procedimento para criar uma senha de acesso ao sistema sem certificado digital 
Novos usuários do sistema 
Usuário que já possuam um certificado digital 
Usuário sem certificado digital 
Usuário já existente com certificado digital 
Usuário cadastrado como Push, sem certificado digital 
Procedimento de secretaria nos tribunais 
Possíveis problemas e respostas prováveis 
Procedimentos para o administrador do sistema 
Habilitação ou restrição da funcionalidade de acesso sem o certificado digital 
Configuração do modelo do texto do termo de compromisso 
Configuração do modelo do texto do envio do link para cadastro da senha 
Exemplo de modelo de texto do e­mail para alteração/cadastro da senha 
Alteração do hash para a troca da senha no banco de dados 
Procedimentos para os desenvolvedores do sistema 
Simulando o acesso com 

### Saving in the vector store